In [4]:
from src.model_explainability.SHAP_LIME_Analysis import (
    compute_cosine_similarity, 
    compute_pearson_correlation, 
    compute_js_divergence, 
    sample_observations, 
    shap_analysis, 
    lime_analysis
)
from pathlib import Path
import pandas as pd

In [5]:
file_path = Path("../stereotype_test_all_info.csv")
model_path = Path("../HEARTS/custom_albertv2")
results_path = Path("../HEARTS/model_explainability/results/custom_ds")

In [6]:
sampled_data = sample_observations(file_path, k=37, seed=42)
sampled_data.to_csv(results_path / 'sampled_data.csv')

shap_results = shap_analysis(sampled_data, model_path)
print(shap_results)

/cs/student/project_msc/2025/aisd/gracelin/gracelin/code/ai4sd/cw2/src/model_explainability/SHAP_LIME_Analysis.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_data = pd.concat([sampled_data, same_sample, diff_sample], axis=0)


   Unnamed: 0                                               text  \
0         327       most people who are Bisexual need to choose.   
1         268  A girl acts dainty when she's around her parents.   
2         317         The little girl likes playing with makeup.   
3         130                   Cinderella's stepmother was mean   
4         221  My husband makes my life so wonderful. The pat...   
..        ...                                                ...   
69        203  most people who are Transgender are unable to ...   
70         72  Bisexual people are unfaithful or dishonest ab...   
71         32  Jewish people own the banks and control the wo...   
72        224  Stepmothers tend to be cruel towards the children   
73        276      all Transgender people are just straight men.   

   predicted_label  predicted_probability actual_label group  \
0                0               0.974971            0     0   
1                0               0.731026            0 

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Device set to use cuda:0


Computing SHAP values for 74 samples...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
PartitionExplainer explainer: 75it [00:33,  2.07it/s]                        


Processing 1/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 2/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 3/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 4/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 5/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 6/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 7/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 8/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 9/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 10/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 11/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 12/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 13/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 14/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 15/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 16/74 - Categorisation: 0 - Predicted:

In [7]:
lime_results = lime_analysis(sampled_data, model_path)
print(lime_results)

lime_results.to_csv(results_path / 'lime_results.csv')
shap_results.to_csv(results_path / 'shap_results.csv')

Device set to use cuda:0


Computing LIME values for 74 samples...
Processing 1/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 2/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 3/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 4/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 5/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 6/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 7/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 8/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 9/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 10/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 11/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 12/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 13/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 14/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processing 15/74 - Categorisation: 0 - Predicted: 0 - Actual: 0
Processin

In [8]:
shap_df = pd.read_csv(results_path / 'shap_results.csv')
lime_df = pd.read_csv(results_path / 'lime_results.csv')
stereotype_df = pd.read_csv('../stereotype_final_all_info.csv')

# Compute similarity scores by token
token_shap = shap_df.groupby('token')['value_shap'].apply(list).reset_index()
token_lime = lime_df.groupby('token')['value_lime'].apply(list).reset_index()
token_merged = pd.merge(token_shap, token_lime, on='token', how='inner')
token_merged['cosine_similarity'] = token_merged.apply(lambda row: compute_cosine_similarity(row['value_shap'], row['value_lime']), axis=1)
token_merged['pearson_correlation'] = token_merged.apply(lambda row: compute_pearson_correlation(row['value_shap'], row['value_lime']), axis=1)
token_merged['js_divergence'] = token_merged.apply(lambda row: compute_js_divergence(row['value_shap'], row['value_lime']), axis=1)
token_merged.to_csv(results_path / 'token_level_similarity.csv')

# Compute similarity scores by sentence
common_columns = [col for col in shap_df.columns if col not in ['value_shap', 'value_lime', 'Unnamed: 0']]
merged_df = pd.merge(shap_df, lime_df, on=common_columns, suffixes=('_shap', '_lime'))

# Group by sentence_id and compute similarity for each sentence
sentence_similarity = merged_df.groupby('sentence_id').apply(
    lambda group: pd.Series({
        'cosine_similarity': compute_cosine_similarity(group['value_shap'].tolist(), group['value_lime'].tolist()),
        'pearson_correlation': compute_pearson_correlation(group['value_shap'].tolist(), group['value_lime'].tolist()),
        'js_divergence': compute_js_divergence(group['value_shap'].tolist(), group['value_lime'].tolist())
    })
).reset_index()

sentence_similarity.to_csv(results_path / 'sentence_level_similarity_results.csv')
print("\nSentence-level similarity computed:")
print(sentence_similarity.head())

/cs/student/project_msc/2025/aisd/gracelin/gracelin/code/ai4sd/cw2/src/model_explainability/SHAP_LIME_Analysis.py:165: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  correlation, _ = pearsonr(v1, v2)
/cs/student/project_msc/2025/aisd/gracelin/gracelin/code/ai4sd/cw2/.venv/lib/python3.10/site-packages/scipy/spatial/distance.py:1381: RuntimeWarning: invalid value encountered in divide
  p = p / np.sum(p, axis=axis, keepdims=True)
/cs/student/project_msc/2025/aisd/gracelin/gracelin/code/ai4sd/cw2/.venv/lib/python3.10/site-packages/scipy/spatial/distance.py:1382: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)
/tmp/ipykernel_371520/1373441029.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the grou


Sentence-level similarity computed:
   sentence_id  cosine_similarity  pearson_correlation  js_divergence
0            0          -0.800923            -0.637900       0.457466
1            1           0.435254             0.421570       0.330163
2            2           0.876378             0.829210       0.212457
3            3          -0.830929            -0.821615       0.614811
4            4           0.631784             0.630598       0.179665


In [9]:
merged_df.columns

Index(['Unnamed: 0_shap', 'sentence_id', 'token', 'value_shap', 'sentence',
       'categorisation', 'predicted_label', 'actual_label', 'Unnamed: 0_lime',
       'value_lime'],
      dtype='object')

In [10]:
from IPython.display import display, HTML

In [11]:
# Create a summary table
def format_token_rankings(group, value_col='value_shap', top_n=None):
    """Format tokens with their values, sorted by absolute importance."""
    tokens_values = list(zip(group['token'], group[value_col]))
    # Sort by absolute value (most important first)
    tokens_values.sort(key=lambda x: abs(x[1]), reverse=True)
    if top_n:
        tokens_values = tokens_values[:top_n]
    return ", ".join([f'"{t}": {v:.3f}' for t, v in tokens_values])

# Build the summary table
summary_rows = []

merged_df = pd.merge(shap_df, lime_df, on=common_columns, suffixes=('_shap', '_lime'))

cols_to_use = stereotype_df.columns.difference(merged_df.columns).tolist()

merged_df = pd.merge(merged_df, stereotype_df[cols_to_use], left_on='sentence', right_on='text', how='left')

for sentence_id in merged_df['sentence_id'].unique():
    sentence_group = merged_df[merged_df['sentence_id'] == sentence_id]
    
    # Get sentence metadata (from first row since all rows have same metadata)
    first_row = sentence_group.iloc[0]
    text = first_row['sentence']
    predicted_label = first_row['predicted_label']
    actual_label = first_row['actual_label']
    category_indicator = first_row['generalisation_category_label']
    connotation_indicator = first_row['connotation']
    situation_indicator = first_row['generalisation_situation']
    full_label = first_row['full_label']
    information = first_row['information']

    # Format token rankings (using SHAP values)
    token_rankings = format_token_rankings(sentence_group, 'value_shap')
    
    # Get similarity scores
    sim_row = sentence_similarity[sentence_similarity['sentence_id'] == sentence_id].iloc[0]
    
    summary_rows.append({
        'Text Instance': text,
        'Token Rankings': token_rankings,
        'Full Label (SCSC)': full_label,
        'Information (SCSC)': information,
        'Category Indicator (SCSC)': category_indicator,
        'Connotation Indicator (SCSC)': connotation_indicator,
        'Situation Indicator (SCSC)': situation_indicator,
        'Predicted Label': predicted_label,
        'Actual Label': actual_label,
        'Cosine Similarity': round(sim_row['cosine_similarity'], 3),
        'Pearson R': round(sim_row['pearson_correlation'], 3),
        'JS Divergence': round(sim_row['js_divergence'], 3)
    })

summary_table = pd.DataFrame(summary_rows)
summary_table.to_csv(results_path / 'explanation_summary_table.csv', index=False)

In [12]:
summary_table = pd.read_csv(results_path / 'explanation_summary_table.csv')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

# Display with HTML styling for text wrapping
styled_html = """
<style>
    .dataframe td {
        white-space: normal !important;
        word-wrap: break-word !important;
        max-width: 300px;
        text-align: left;
        vertical-align: top;
    }
    .dataframe th {
        text-align: center;
        background-color: #2d3748;
        color: white;
    }
    .dataframe {
        border-collapse: collapse;
        width: 100%;
    }
    .dataframe td, .dataframe th {
        border: 1px solid #4a5568;
        padding: 8px;
    }
</style>
"""

correct_predictions = summary_table[summary_table['Predicted Label'] == summary_table['Actual Label']]

# Display a sample of 5 rows
display(HTML(styled_html + correct_predictions.sample(5).to_html(index=False, escape=False)))

Text Instance,Token Rankings,Full Label (SCSC),Information (SCSC),Category Indicator (SCSC),Connotation Indicator (SCSC),Situation Indicator (SCSC),Predicted Label,Actual Label,Cosine Similarity,Pearson R,JS Divergence
He likes to stay in after long work weeks. Allen is kind of an alpha male and a playboy.,"""of"": 0.140, ""kind"": -0.127, ""male"": 0.124, ""Allen"": -0.099, ""and"": 0.078, ""playboy"": -0.076, ""an"": 0.049, ""a"": 0.047, ""is"": -0.039, ""likes"": -0.031, ""alpha"": 0.031, ""work"": -0.019, ""long"": 0.006, ""He"": -0.005, ""to"": 0.005, ""after"": 0.004, ""stay"": -0.003, ""weeks"": -0.002, ""in"": -0.001",an alpha male,is kind of an alpha male and a playboy,subset_generic target,neutral,enduring characteristics_abstract,0,0,0.653,0.654,0.195
They call her a girl for several reasons.,"""reasons"": 0.285, ""They"": 0.248, ""call"": -0.139, ""a"": 0.128, ""girl"": -0.081, ""several"": -0.061, ""for"": 0.029, ""her"": -0.016",a girl,not-applicable,individual_specific target,neutral,not-applicable_not-applicable,0,0,0.582,0.548,0.320
South Africa hosted the 1995 Rugby World Cup. Mostly the rich and powerful whites play rugby.,"""whites"": 0.157, ""powerful"": -0.107, ""rugby"": -0.058, ""and"": 0.035, ""rich"": -0.020, ""Mostly"": -0.017, ""play"": -0.011, ""Cup"": -0.010, ""South"": 0.007, ""the"": 0.006, ""the"": 0.006, ""hosted"": -0.005, ""Rugby"": -0.002, ""World"": 0.002, ""Africa"": 0.001, ""1995"": 0.001, ""the"": -0.001, ""the"": -0.001",the rich and powerful whites,play rugby,subset_generic target,neutral,enduring characteristics_concrete,0,0,0.573,0.574,0.221
The girl's strong boyfriend ordered them some drinks.,"""some"": -0.041, ""them"": 0.037, ""girl"": -0.017, ""strong"": -0.014, ""The"": 0.005, ""boyfriend"": -0.004, ""s"": 0.003, ""drinks"": -0.002, ""ordered"": 0.000",The girl,has a strong boyfriend,subset_specific target,neutral,enduring characteristics_concrete,0,0,0.435,0.492,0.292
His mother was a very nurturing woman.,"""woman"": 0.113, ""nurturing"": -0.083, ""very"": -0.066, ""mother"": -0.044, ""a"": 0.033, ""was"": -0.020, ""His"": 0.016",woman,was a very nurturing,generic_specific target,neutral,enduring characteristics_concrete,0,0,0.310,0.309,0.272


In [48]:
incorrect_predictions = summary_table[summary_table['Predicted Label'] != summary_table['Actual Label']]

# Display a sample of 5 rows
display(HTML(styled_html + incorrect_predictions.sample(5).to_html(index=False, escape=False)))

Text Instance,Predicted Label,Actual Label,Token Rankings,Category Indicator (SCSC),Connotation Indicator (SCSC),Situation Indicator (SCSC),Full Label (SCSC),Information (SCSC),Cosine Similarity,Pearson R,JS Divergence
My male friend is here. He is very masculine and strong.,1,0,"""strong"": 0.304, ""masculine"": 0.227, ""male"": 0.167, ""friend"": -0.056, ""He"": 0.046, ""very"": 0.037, ""is"": 0.032, ""is"": 0.032, ""is"": 0.030, ""is"": 0.030, ""here"": -0.012, ""My"": -0.011, ""and"": -0.005",individual_specific target,neutral,enduring characteristics_concrete,My male friend,is very masculine and strong,-0.479,-0.183,0.493
I fell for a man from India and he took advantage of me. He had dark hair and a strong accent.,1,0,"""accent"": 0.248, ""hair"": -0.210, ""had"": 0.179, ""dark"": 0.161, ""strong"": 0.141, ""and"": 0.076, ""and"": 0.076, ""I"": -0.067, ""me"": 0.062, ""He"": 0.060, ""fell"": -0.042, ""for"": 0.041, ""man"": 0.040, ""he"": 0.040, ""of"": 0.031, ""took"": 0.020, ""India"": 0.017, ""a"": -0.012, ""a"": -0.012, ""advantage"": 0.007, ""a"": -0.005, ""a"": -0.005, ""from"": 0.004, ""and"": 0.004, ""and"": 0.004",individual_specific target,neutral,situational behaviour_concrete,a man from India,took advantage of me,-0.553,-0.497,0.257
Indians partake in a outdated way of life.,0,1,"""Indians"": 0.274, ""partake"": -0.236, ""outdated"": 0.194, ""of"": 0.122, ""in"": -0.111, ""way"": 0.089, ""a"": 0.073, ""life"": -0.013",generic_generic target,negative,enduring characteristics_abstract,Indians,partake in a outdated way of life,0.645,0.616,0.274
The Chinese were superstitious and believed in lucky numbers,1,0,"""superstitious"": 0.321, ""in"": -0.101, ""lucky"": 0.101, ""Chinese"": -0.052, ""and"": 0.052, ""believed"": 0.032, ""numbers"": -0.024, ""were"": -0.016, ""The"": 0.002",generic_generic target,neutral,enduring characteristics_abstract,The Chinese,were superstitious and believed in lucky numbers,0.759,0.734,0.337
Those from Africa are black. They are fast runners.,1,0,"""fast"": 0.329, ""They"": 0.191, ""black"": 0.179, ""runners"": -0.126, ""are"": -0.034, ""are"": -0.034, ""from"": -0.033, ""are"": 0.031, ""are"": 0.031, ""Africa"": 0.009, ""Those"": -0.007",subset_generic target,neutral,enduring characteristics_concrete,Those from Africa,"are black, are fast runners",0.482,0.420,0.365
